<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/9999.%20Ejercicios" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soluciones a los ejercicios cuaderno **3: Output parsers**
---



# **Preparando el entorno del cuaderno**
---

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU

# Instalamos Rich para mejorar la salida
%pip install rich -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos las librerias para formatear mejor la salida
from IPython.display import Markdown, display
from rich import print as rprint

# **👨🏻‍🏫 Ejercicio 1: Extractor de Información de Películas**

**Objetivo**: Crear un sistema que extraiga información estructurada sobre películas a partir de una consulta simple.

**Descripción**: Implementa un programa utilizando LangChain y `StructuredOutputParser` que tome como entrada el nombre de una película y genere una ficha técnica con información relevante como título, director, género, año, puntuación y resumen.

**Requisitos**:

1. Utilizar `ResponseSchema` para definir la estructura de los campos a extraer
2. Implementar un `StructuredOutputParser` para procesar la respuesta del modelo
3. Crear un `PromptTemplate` que incluya las instrucciones de formato
4. Integrar el sistema con un modelo de lenguaje (ChatOpenAI o ChatGroq)
5. Procesar y mostrar la información de manera estructurada

**Entregable**: Un script de Python que, al ejecutarse con el nombre de una película como entrada, genere una ficha técnica estructurada con la información solicitada.

**Nivel**: Medio

**Aplicación práctica**: Este sistema puede ser útil para crear bases de datos de películas, generar fichas técnicas automáticas o construir sistemas de recomendación basados en atributos específicos de las películas.

### Resuelve el ejerciccio en este cuaderno. Una solucion la puedes encontrar en el repositorio de esta serie de cuadernos.
https://github.com/juanfranbrv/curso-langchain

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
esquema_respuesta = [
            ResponseSchema(name="titulo", description="Titulo de la película."),
            ResponseSchema(name="director", description="Nombre del director de la película"),
            ResponseSchema(name="genero", description="Genero cinematografico de  la pelicula"),
            ResponseSchema(name="año", description="Año de estreno de la película"),
            ResponseSchema(name="puntuacion", description="Número de estrellas 1,2,3,4,5"),
            ResponseSchema(name="resumen", description="Resumen de la crítica de la película")
                    ]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(esquema_respuesta)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
plantilla = """
        Genera la siguiente informacion sobre esta {pelicula}:
            Titulo
            Director
            Genero
            Puntuacion

        solo devuelve la informacion solicitada en el formato indicado


        {format_instructions}
        """

# Generar el prompt para el LLM
prompt_template = PromptTemplate(
    template=plantilla,
    input_variables=["pelicula"],
    partial_variables={"format_instructions": format_instructions}
)

pelicula = "El señor de los anillos"
prompt = prompt_template.format(pelicula=pelicula)


# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0.7)

respuesta = modelo.invoke(prompt).content

# Parseamos la respuesta con el parser de salida
respuesta_formateada = output_parser.parse(respuesta)

print(respuesta)
rprint(type(respuesta))
rprint("---")
rprint(respuesta_formateada)
rprint(type(respuesta_formateada))



```json
{
	"titulo": "El Señor de los Anillos",
	"director": "Peter Jackson",
	"genero": "Aventura, Fantasía",
	"año": "2001",
	"puntuacion": "5",
	"resumen": "Una adaptación épica y emocionante de la novela de J.R.R. Tolkien, con una narrativa rica y personajes bien desarrollados."
}
```


<class 'str'>

---

{
    'titulo': 'El Señor de los Anillos',
    'director': 'Peter Jackson',
    'genero': 'Aventura, Fantasía',
    'año': '2001',
    'puntuacion': '5',
    'resumen': 'Una adaptación épica y emocionante de la novela de J.R.R. Tolkien, con una narrativa rica y 
personajes bien desarrollados.'
}

<class 'dict'>

# 👨🏻‍🏫 **Ejercicio: Extractor de información personal de un texto**

**Objetivo**:  
Crear un programa que **extraiga información estructurada** de un texto utilizando `with_structured_output()`. La información a extraer incluye:

- Nombre completo
    
- Edad (número entero)
    
- Población (ciudad o país)
    
- Correo electrónico
    
- Lista de información adicional


**Requerimientos**:

1. **Esquema Pydantic**:  
    Define una clase `InformacionExtraida` usando `pydantic.BaseModel` con los campos solicitados y sus descripciones.
    
2. **Prompt Template**:  
    Crea un prompt que indique al modelo de lenguaje qué información extraer y en qué formato.
    
3. **Modelo con salida estructurada**:  
    Usa `with_structured_output()` para configurar el modelo y garantizar que la salida cumpla con el esquema definido.
    
4. **Pruebas**:  
    Ejecuta el programa con el texto de ejemplo y muestra los resultados estructurados.

### Resuelve el ejercicio en este cuaderno. Una solucion la puedes encontrar en el repositorio de esta serie de cuadernos.
https://github.com/juanfranbrv/curso-langchain

In [ ]:
from typing import List
from pydantic import BaseModel, Field


class InformacionExtraida(BaseModel):
    nombre: str = Field(description="Nombre completo de la persona")
    edad: int = Field(description="Edad de la persona")
    poblacion: str = Field(description="Ciudad o país de residencia")
    correo: str = Field(description="Correo electrónico válido")
    otros: List[str] = Field(description="Lista de información adicional relevante")


plantilla = """
Extrae esta información del texto:
- Nombre completo
- Edad (número entero)
- Población (ciudad/país)
- Correo electrónico
- Otra información relevante (lista)

Texto: {texto}

Solo devuelve la información solicitada, nada más.
"""

prompt_template = PromptTemplate(
    template=plantilla,
    input_variables=["texto"]
)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)
# modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0.7)

modelo_struct = modelo.with_structured_output(InformacionExtraida)


texto_ejemplo = """
Hola, soy Carlos Pérez, tengo 28 años.
Vivo en Buenos Aires y mi email es carlos@example.com.
Trabajo como ingeniero y me gusta el fútbol.
"""

prompt = prompt_template.format(texto=texto_ejemplo)
resultado = modelo_struct.invoke(prompt)

rprint(resultado)
rprint(type(resultado))
print("\n\n")


# Uso del resultado estructurado
print(f"Nombre: {resultado.nombre}")
print(f"Edad: {resultado.edad}")
print(f"Población: {resultado.poblacion}")
print(f"Correo: {resultado.correo}")
print("Otros:")
for otro in resultado.otros:
    print(f"  - {otro}")


InformacionExtraida(nombre='Pedro', edad=0, poblacion='', correo='pedro@test.com', otros=[])

<class '__main__.InformacionExtraida'>




Nombre: Pedro
Edad: 0
Población: 
Correo: pedro@test.com
Otros:


# 👨🏻‍🏫  **Ejercicio: Extracción de Recetas Estructuradas**

#### **Objetivo**

Crear un script en Python que:

1. Extraiga el texto de una página web de recetas.
    
2. Utilice un modelo de lenguaje (LLM) para estructurar la información en un formato específico.
    
3. Muestre los resultados de forma organizada usando Pydantic.


### **Requerimientos**

1. **Extracción de texto web**:
    
    - Usar `requests` y `BeautifulSoup` para obtener el texto crudo de una URL de receta.
        
        
2. **Modelado de datos con Pydantic**:
    
    - Definir 3 clases:
        
        - `Ingrediente` (nombre y cantidad).
            
        - `Paso` (número y descripción).
            
        - `Receta` (título, lista de ingredientes, lista de pasos).
            
3. **Prompt Engineering**:
    
    - Crear un prompt que indique al LLM extraer: título, ingredientes y pasos de la receta.
        
4. **Configuración del LLM**:
    
    - Usar `with_structured_output()` para garantizar que la salida del modelo coincida con el esquema `Receta`.
        
5. **Pruebas y visualización**:
    
    - Mostrar el resultado estructurado y acceder a sus campos (ej: `resultado.titulo`).

In [ ]:
%pip install requests beautifulsoup4 -qU

import requests
from bs4 import BeautifulSoup

url = "https://www.bonviveur.es/recetas/coles-de-bruselas-en-freidora-de-aire"
url = "https://www.directoalpaladar.com/postres/tarta-mango-postre-perfecto-para-cualquier-ocasion"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
texto_raw= soup.get_text()

# =====================

from typing import List
from pydantic import BaseModel, Field

class Ingrediente(BaseModel):
    nombre: str = Field(description="Nombre del ingrediente")
    cantidad: str = Field(description="Cantidad del ingrediente")

class Paso(BaseModel):
    numero: int = Field(description="Número del paso")
    descripcion: str = Field(description="Descripción del paso")

class Receta(BaseModel):
    titulo: str = Field(description="Título de la receta")
    ingredientes: List[Ingrediente] = Field(description="Lista de ingredientes")
    pasos: List[Paso] = Field(description="Lista de pasos")

plantilla = """
Extrae esta información de la receta:
- Título
- Ingredientes (lista de objetos con nombre y cantidad)
- Pasos (lista de objetos con número y descripción)

Receta: {texto}
"""

prompt_template = PromptTemplate(
    template=plantilla,
    input_variables=["texto"]
)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)
# modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0.7)


modelo_struct = modelo.with_structured_output(Receta)

prompt = prompt_template.format(texto=texto_raw)
resultado = modelo_struct.invoke(prompt)

rprint(resultado)
rprint(type(resultado))
print("\n\n")


Receta(
    titulo='Tarta de mango, un postre perfecto para cualquier ocasión',
    ingredientes=[
        Ingrediente(nombre='Harina de trigo', cantidad='150 g'),
        Ingrediente(nombre='Levadura química', cantidad='12 g'),
        Ingrediente(nombre='Canela molida', cantidad='1 cucharadita'),
        Ingrediente(nombre='Azúcar', cantidad='125 g'),
        Ingrediente(nombre='Yogur griego', cantidad='1'),
        Ingrediente(nombre='Aceite de oliva', cantidad='175 g'),
        Ingrediente(nombre='Sirope de arce', cantidad='1 cucharada'),
        Ingrediente(nombre='Esencia de vainilla', cantidad='Una pizca'),
        Ingrediente(nombre='Sal', cantidad='Una pizca'),
        Ingrediente(nombre='Huevos', cantidad='3'),
        Ingrediente(nombre='Mango', cantidad='2'),
        Ingrediente(nombre='Limón (ralladura)', cantidad='1'),
        Ingrediente(nombre='Queso crema (para el frosting)', cantidad='200 g'),
        Ingrediente(nombre='Azúcar glas (para el frosting)', cantidad='50 g'),
        Ingrediente(nombre='Zumo de limón (para el frosting)', cantidad='1 cucharada')
    ],
    pasos=[
        Paso(numero=1, descripcion='Mezclar la harina con el azúcar, la canela, la levadura y la sal.'),
        Paso(
            numero=2,
            descripcion='En otro cuenco batir los huevos con el aceite de oliva, el yogur, el sirope de arce y la 
esencia de vainilla.'
        ),
        Paso(numero=3, descripcion='Agregar la ralladura de medio mango y la de limón, y remover.'),
        Paso(numero=4, descripcion='Incorporar la harina a esta mezcla y remover bien.'),
        Paso(
            numero=5,
            descripcion='Engrasar un molde redondo desmontable con un poco de mantequilla y enharinarlo.'
        ),
        Paso(
            numero=6,
            descripcion='Verter la mezcla del bizcocho en el molde y hornear durante 45 minutos a 175ºC.'
        ),
        Paso(
            numero=7,
            descripcion='Comprobar la cocción del bizcocho con un palillo, dejar enfriar y desmoldar.'
        ),
        Paso(numero=8, descripcion='Preparar el frosting mezclando los ingredientes y reservar.'),
        Paso(numero=9, descripcion='Cortar el bizcocho en dos discos gruesos.'),
        Paso(
            numero=10,
            descripcion='Cubrir con el frosting el primer disco, agregar dados de mango y cerrar con la otra capa 
de bizcocho.'
        ),
        Paso(numero=11, descripcion='Cubrir con otra capa de frosting y decorar con mango troceado.')
    ]
)

<class '__main__.Receta'>